<a href="https://colab.research.google.com/github/Koks-creator/OcrBenthamDataset/blob/main/yolov11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


In [ ]:
!unzip /mydrive/yoloTrain/train_data.zip -d /content

Archive:  /mydrive/yoloTrain/train_data.zip
   creating: /content/train_data/
   creating: /content/train_data/images/
   creating: /content/train_data/images/train/
  inflating: /content/train_data/images/train/gunsondupaxd_0_1.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_10.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_100.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_101.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_102.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_103.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_104.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_105.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_106.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_107.jpg  
  inflating: /content/train_data/images/train/gunsondupaxd_0_108.jpg  
  inflating: /content/train_data/images/train/gunsondupa

In [ ]:
!pip install ultralytics==8.3.89 sahi==0.11.22

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.6/932.6 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88


In [ ]:
!pip freeze | grep -e ultralytics -e sahi -e torch

sahi==0.11.22
torch==2.8.0+cu126
torchao==0.10.0
torchaudio==2.8.0+cu126
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.23.0+cu126
ultralytics==8.3.89
ultralytics-thop==2.0.18


In [ ]:
from ultralytics import YOLO
from sahi.utils.ultralytics import download_yolo11n_model
import yaml
import os
import torch
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    device_name = torch.cuda.get_device_name(0) if device_count > 0 else "None"
    print(f"Found {device_count} GPU devices.")
    print(f"Active device: {device_name}")
    device = 0  # Użyj pierwszego GPU
else:
    print("GPU not found, using CPU.")
    device = 'cpu'

Found 1 GPU devices.
Active device: Tesla T4


In [ ]:
DATASET_PATH = "/content/train_data"
CLASS_NAMES = ["line"]
MODEL_DIR = "./models"
MODEL_NAME = "yolo11m.pt"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME)
BATCH_SIZE = 16
WORKERS = 8
CONFIG_YAML_FILE = "./config/data.yaml"

In [ ]:
os.makedirs(MODEL_DIR, exist_ok=True)

# if not os.path.exists(MODEL_PATH):
#     print(f"Model {MODEL_NAME} not found. Downloading...")

#     download_yolo11n_model(MODEL_PATH)
#     print(f"Model has been downloaded, to {MODEL_PATH}")

In [ ]:
data_yaml = {
    'path': DATASET_PATH,
    'train': 'images/train',
    'val': 'images/val',
    'names': {i: name for i, name in enumerate(CLASS_NAMES)},
    'nc': len(CLASS_NAMES)
}

os.makedirs("config", exist_ok=True)
with open(CONFIG_YAML_FILE, 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"Config file created: {CONFIG_YAML_FILE}")

Config file created: ./config/data.yaml


In [ ]:
print(f"Loading model from {MODEL_PATH}...")
model = YOLO(MODEL_PATH)

Loading model from ./models/yolo11m.pt...


100%|██████████| 38.8M/38.8M [00:03<00:00, 12.8MB/s]


In [ ]:
MODEL_PATH

'./models/yolo11m.pt'

In [ ]:
!ls

config	gdrive	models	sample_data  train_data


In [ ]:
# test vanila paramy
results = model.train(
    data=CONFIG_YAML_FILE,
    epochs=100,
    imgsz=800,
    batch=BATCH_SIZE,
    project="runs",
    name="yolov11_text_lines",
    save=True,
    device=device,
    workers=WORKERS
)

New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.89 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=./models/yolo11m.pt, data=./config/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=yolov11_text_lines, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,

100%|██████████| 755k/755k [00:00<00:00, 135MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 337MB/s]


AMP: checks passed ✅


...

    100/100        13G     0.7404     0.4169     0.8346        391        800: 100%|██████████| 25/25 [00:23<00:00,  1.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.65it/s]

                   all         10        277      0.978      0.976      0.992      0.733



100 epochs completed in 0.713 hours.
Optimizer stripped from runs/yolov11_text_lines/weights/last.pt, 40.5MB
Optimizer stripped from runs/yolov11_text_lines/weights/best.pt, 40.5MB

Validating runs/yolov11_text_lines/weights/best.pt...
Ultralytics 8.3.89 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.16it/s]


                   all         10        277      0.968      0.974      0.991       0.74
Speed: 0.4ms preprocess, 14.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/yolov11_text_lines


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [ ]:
# results = model.train(
#     data=CONFIG_YAML_FILE,
#     epochs=80,
#     imgsz=800,
#     batch=BATCH_SIZE,
#     patience=30,
#     project="runs",
#     name="yolov11_custom",
#     pretrained=True,
#     optimizer='SGD',
#     momentum=0.937,
#     weight_decay=0.0005,
#     lr0=0.0015,       # More moderate learning rate - balance between stability and recall improvement
#     lrf=0.005,        # Gentler LR decay - less aggressive
#     warmup_epochs=5,  # Shorter warmup - faster transition to full LR
#     cos_lr=True,
#     augment=True,
#     mosaic=0.7,       # Less intense mosaic - cracks may disappear with too strong mosaic
#     mixup=0.15,       # Less intense mixup - preserves more distinct crack features
#     copy_paste=0.2,   # Remains high - good for rare classes
#     degrees=8.0,      # Reduced rotation - cracks may have directional characteristics
#     scale=0.6,        # Larger scale - various crack sizes
#     translate=0.2,    # Added translation - helps with different crack locations
#     fliplr=0.5,       # Added horizontal flip - increases data diversity
#     hsv_h=0.015,
#     hsv_s=0.7,
#     hsv_v=0.4,
#     perspective=0.0005,
#     shear=1.0,        # Reduced shearing - preserves natural appearance of cracks
#     cls=1.0,          # Higher weight for classification - crucial for recall
#     box=3.5,          # Lower weight for box loss - less emphasis on location accuracy
#     iou=0.35,         # Lower IoU threshold - increases recall at the cost of precision
#     save=True,
#     save_period=5,    # More frequent model saving
#     device=device,
#     workers=WORKERS
# )

results = model.train(
  # Zoptymalizowana konfiguracja dla wykrywania ludzi i broni
  data=CONFIG_YAML_FILE,
  epochs=100,              # Mniej epok niż dla owadów - ludzie i broń mają wyraźniejsze cechy
  imgsz=640,               # Wysoka rozdzielczość dla małych broni
  batch=BATCH_SIZE,
  patience=25,             # Krótszy czas zbieżności dla wyraźniejszych obiektów
  project="runs",
  name="yolov11_person_gun",
  pretrained=True,
  optimizer='AdamW',       # Dobry wybór dla zadań detekcji
  momentum=0.937,
  weight_decay=0.0005,     # Zwiększone, by zapobiec przeuczeniu się na konkretnych wyglądach broni
  lr0=0.001,               # Niższy początkowy learning rate
  lrf=0.01,                # Zachowany wyższy końcowy LR
  warmup_epochs=3,
  cos_lr=True,
  augment=True,

  # Augmentacje dostosowane do ludzi i broni
  mosaic=0.6,              # Wyższe - pomaga z różnymi pozycjami broni
  mixup=0.2,               # Niższe - zachowuje charakterystyczne cechy broni
  copy_paste=0.4,          # Wyższe - pomaga z balansem klas
  degrees=20.0,            # Niższe - ludzie zazwyczaj w pionie, broń ma typowe orientacje
  scale=0.6,               # Umiarkowany zakres skali
  translate=0.2,           # Umiarkowane przesunięcia
  fliplr=0.5,              # Odbicie poziome OK
  flipud=0.0,              # Bez odbicia pionowego - ludzie i broń mają orientację góra/dół
  hsv_h=0.015,             # Mniejsza wariacja odcienia - realistyczne kolory
  hsv_s=0.7,               # Większa wariacja nasycenia - broń w różnym oświetleniu
  hsv_v=0.4,               # Umiarkowana wariacja jasności
  perspective=0.0005,      # Lekka perspektywa - broń widziana pod różnymi kątami
  shear=1.0,               # Niższe - zachowuje charakterystyczne kształty

  # Wagi loss dla obiektów o różnych rozmiarach
  cls=0.3,                 # Niższa waga klasyfikacji - mniej klas do rozróżnienia
  box=8.0,                 # Wyższa waga box loss - kluczowa dla dokładnej detekcji broni
  dfl=1.5,                 # Distribution Focal Loss dostosowany do różnorodnych rozmiarów

  # Parametry detekcji
  conf=0.25,               # Umiarkowany próg pewności
  iou=0.7,                 # Umiarkowany IoU - lepszy dla nakładających się obiektów (np. osoba z bronią)
  max_det=100,             # Zwiększone - wiele osób i broni może być na jednej klatce

  # Zapisywanie
  save=True,
  save_period=5,           # Częstsze zapisywanie
  device=device,
  workers=WORKERS,

  # Dodatkowe optymalizacje
  amp=True,                # Mixed precision dla szybszego trenowania
  profile=False,
  verbose=True
)

New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.89 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=./models/yolo11m.pt, data=./config/data.yaml, epochs=100, time=None, patience=25, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs, name=yolov11_person_gun, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.25, iou=0.7, max_det=100, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning /content/train_data/labels/train.cache... 209 images, 0 backgrounds, 0 corrupt: 100%|██████████| 209/209 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/train_data/labels/val.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]


Plotting labels to runs/yolov11_person_gun/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)


KeyboardInterrupt: 

In [ ]:
model = YOLO('/content/runs/yolov11_person_gun/weights/best.pt')
# model = YOLO('/mydrive/yoloTrain/best (19).pt')
results = model.train(
    data=CONFIG_YAML_FILE,
    epochs=30,                   # Krótszy trening, skupiony na małych obiektach
    imgsz=896,                   # Znacznie większa rozdzielczość dla oddalonych broni
    batch=max(4, BATCH_SIZE//4), # Mniejszy batch ze względu na większy rozmiar obrazu
    patience=15,
    project="runs",
    name="yolov11_gun_detection_improved",

    # Zrównoważone wagi - większy nacisk na klasyfikację
    cls=1.0,                     # Zwiększona waga klasyfikacji (z 0.4)
    box=5.0,                     # Zmniejszona waga box (z 7.5)
    dfl=1.5,

    # Detekcja małych obiektów
    conf=0.15,                   # Niższy próg pewności dla zwiększenia recall
    iou=0.5,                     # Niższy IoU dla lepszego wykrywania małych obiektów

    # Augmentacje dla symulacji oddalonych broni
    augment=True,
    mosaic=0.6,                  # Zwiększony mosaic dla większej liczby małych obiektów
    scale=0.5,                   # Szerszy zakres skalowania - symulacja różnych odległości

    # Specjalne parametry dla małych obiektów
    overlap_mask=True,           # Lepsze nakładanie masek dla małych obiektów
    perspective=0.001,           # Lekka perspektywa

    # Optymalizator
    optimizer='AdamW',
    lr0=0.0005,
    lrf=0.002,
    weight_decay=0.0001,

    # Zapobieganie przeuczeniu
    dropout=0.1,                 # Dodanie dropoutu

    # Pozostałe parametry
    device=device,
    workers=WORKERS,
    amp=True,
    verbose=True,
    save_period=5,
    plots=True
)

New https://pypi.org/project/ultralytics/8.3.192 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.89 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/yolov11_person_gun/weights/best.pt, data=./config/data.yaml, epochs=30, time=None, patience=15, batch=4, imgsz=896, save=True, save_period=5, cache=False, device=0, workers=8, project=runs, name=yolov11_gun_detection_improved, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=0.15, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina

train: Scanning /content/train_data/labels/train.cache... 881 images, 4 backgrounds, 0 corrupt: 100%|██████████| 885/885 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/train_data/labels/val.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]


Plotting labels to runs/yolov11_gun_detection_improved/labels.jpg... 
optimizer: AdamW(lr=0.0005, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 896 train, 896 val
Using 2 dataloader workers
Logging results to runs/yolov11_gun_detection_improved
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      30/30      4.53G     0.6905       1.44      1.446          1        896: 100%|██████████| 222/222 [00:40<00:00,  5.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00, 10.49it/s]

                   all         19         73      0.885       0.72      0.839      0.624



30 epochs completed in 0.366 hours.
Optimizer stripped from runs/yolov11_gun_detection_improved/weights/last.pt, 19.2MB
Optimizer stripped from runs/yolov11_gun_detection_improved/weights/best.pt, 19.2MB

Validating runs/yolov11_gun_detection_improved/weights/best.pt...
Ultralytics 8.3.89 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:17<00:00,  5.91s/it]


                   all         19         73      0.916      0.713      0.836      0.621
                person         11         32      0.903      0.781      0.874      0.676
                   gun         19         41       0.93      0.644      0.798      0.566
Speed: 0.6ms preprocess, 928.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/yolov11_gun_detection_improved


# YOLOv11 Training Parameters
https://docs.ultralytics.com/modes/train/#train-settings
<br>
https://www.ultralytics.com/glossary/data-augmentation


# YOLOv11 Training Parameters with Practical Guidelines

| Argument | Type | Default | Description | Practical Guideline |
|----------|------|---------|-------------|---------------------|
| `model` | str | None | Specifies the model file for training. Accepts a path to either a .pt pretrained model or a .yaml configuration file. | For most tasks, use a pretrained YOLOv11 model (e.g., 'yolov11n.pt') for transfer learning rather than training from scratch. |
| `data` | str | None | Path to the dataset configuration file (e.g., coco8.yaml) containing dataset-specific parameters. | Create a custom YAML file with your dataset paths and class names; follow the COCO format for best compatibility. |
| `epochs` | int | 100 | Total number of training epochs, each representing a full pass over the dataset. | Use 50-100 for transfer learning; 100-300 for training from scratch; monitor validation metrics to avoid overfitting. |
| `time` | float | None | Maximum training time in hours, overriding the epochs argument. | Useful for time-boxed experiments; set to 24-48 hours for complex datasets when exact epoch count is less important than time constraints. |
| `patience` | int | 100 | Number of epochs to wait without improvement before early stopping. | Set to 20-50 for most tasks; lower (10-15) for quick experiments; higher (50-100) for complex datasets with slow convergence. |
| `batch` | int | 16 | Batch size for training (integer value, -1 for auto-sizing, or fraction for GPU utilization). | Use largest value that fits in memory; 8-16 for 8GB GPU; 32-64 for 24GB+ GPUs; smaller batches (4-8) can improve generalization. |
| `imgsz` | int or list | 640 | Target image size for training, affecting model accuracy and speed. | Use 640-1280 for general detection; larger (1280+) for small objects; smaller (416-512) for speed-critical applications. |
| `save` | bool | True | Enables saving of training checkpoints and final model weights. | Always keep True unless running purely experimental tests where saving is unnecessary. |
| `save_period` | int | -1 | Frequency of saving model checkpoints in epochs. | Set to 10-20 for long trainings to capture model evolution; lower (5) for critical/expensive trainings. |
| `cache` | bool | False | Enables caching of dataset images (True/ram, disk, or False). | Use 'disk' for large datasets; True/ram for small datasets (<5GB) with sufficient RAM; False for most typical cases. |
| `device` | int or str or list | None | Specifies computational device(s) for training. | For single GPU use '0'; multi-GPU use [0,1,2]; CPU-only use 'cpu'; use 'mps' for Apple Silicon. |
| `workers` | int | 8 | Number of worker threads for data loading per GPU. | Set to CPU core count minus 2; reduce to 0-4 if experiencing memory issues; increase to 12-16 for high-speed storage systems. |
| `project` | str | None | Name of the project directory for saving training outputs. | Use descriptive names for specific tasks ("traffic-detection", "crack-segmentation") to organize different research directions. |
| `name` | str | None | Name of the training run, creating a subdirectory within the project folder. | Use versioned names ("v1", "small-batch", "high-lr") that describe the specific training configuration for easy comparison. |
| `exist_ok` | bool | False | Allows overwriting of an existing project/name directory. | Set True during active development; False for production to prevent accidental overwriting of important results. |
| `pretrained` | bool or str | True | Determines whether to start training from a pretrained model. | Use True for most cases; False only when training on very specialized data unlike any pretraining dataset. |
| `optimizer` | str | 'auto' | Choice of optimizer for training. | Use 'SGD' for best generalization; 'Adam' for faster convergence; 'auto' works well for standard cases. |
| `seed` | int | 0 | Sets the random seed for training reproducibility. | Change only when comparing different architectures to ensure fair comparison with identical initialization. |
| `deterministic` | bool | True | Forces deterministic algorithm use, ensuring reproducibility. | Keep True for research and benchmarking; can set False for slight speed improvements in production training. |
| `single_cls` | bool | False | Treats all classes as a single class during training. | Set True only for presence/absence detection problems; False for all multi-class detection tasks. |
| `classes` | list[int] | None | Specifies class IDs to train on. | Use to focus on specific classes in a larger dataset; helpful for transfer learning on a subset of original classes. |
| `rect` | bool | False | Enables rectangular training, optimizing batch composition. | Set True for inference speed optimization; keep False during main training for better data augmentation. |
| `multi_scale` | bool | False | Enables multi-scale training by varying image size during training. | Enable for models that need robust multi-scale detection (traffic, aerial); unnecessary for fixed-distance applications. |
| `cos_lr` | bool | False | Utilizes a cosine learning rate scheduler. | Set True for most trainings; provides smoother LR decay and often better final convergence than step-based schedules. |
| `close_mosaic` | int | 10 | Disables mosaic augmentation in the last N epochs. | Set to 5-10% of total epochs; critical for stabilizing final training phase and improving precision. |
| `resume` | bool | False | Resumes training from the last saved checkpoint. | Use when training was interrupted; ensures consistent continuation of optimization process. |
| `amp` | bool | True | Enables Automatic Mixed Precision training. | Keep True on modern GPUs for memory savings and speed; set False only if experiencing numerical stability issues. |
| `fraction` | float | 1.0 | Fraction of the dataset to use for training. | Use 0.1-0.2 for quick experiments; 0.5 for prototype development; 1.0 for final training. |
| `profile` | bool | False | Enables ONNX and TensorRT speed profiling. | Enable only when planning to deploy with these frameworks to identify potential optimization bottlenecks. |
| `freeze` | int or list | None | Freezes the first N layers or specified layer indices. | Freeze 0-10 backbone layers when fine-tuning; more layers (10-15) for very similar datasets to the pretrained one. |
| `lr0` | float | 0.01 | Initial learning rate. | Use 0.01 for SGD with new models; 0.001-0.005 for fine-tuning; 0.0001-0.0005 for delicate final tuning. |
| `lrf` | float | 0.01 | Final learning rate as a fraction of initial rate. | Set 0.01-0.1 for typical training; lower (0.001-0.01) for longer trainings for smoother decay. |
| `momentum` | float | 0.937 | Momentum factor for SGD or beta1 for Adam. | Keep at 0.9-0.95 for most cases; lower (0.8-0.9) for noisy data; higher (0.95-0.99) for stable, well-behaved loss landscapes. |
| `weight_decay` | float | 0.0005 | L2 regularization factor, penalizing large weights. | Use 0.0005-0.001 for most cases; increase to 0.001-0.005 for small datasets prone to overfitting. |
| `warmup_epochs` | float | 3.0 | Number of epochs for learning rate warmup. | Set to 3-5 for stable training start; higher (5-10) for very large batch sizes or high learning rates. |
| `warmup_momentum` | float | 0.8 | Initial momentum for warmup phase. | Keep at default 0.8 for most cases; rarely needs adjustment. |
| `warmup_bias_lr` | float | 0.1 | Learning rate for bias parameters during warmup. | Keep at default 0.1; higher values (0.2-0.5) can sometimes improve initial detection of rare classes. |
| `box` | float | 7.5 | Weight of the box loss component in the loss function. | Use 5.0-7.5 for accurate localization; higher (8.0-10.0) when precise boundaries are critical; lower (3.0-5.0) to prioritize recall. |
| `cls` | float | 0.5 | Weight of the classification loss in the total loss function. | Increase to 1.0-2.0 to improve classification accuracy; decrease to 0.3-0.5 when localization is more important than classification. |
| `dfl` | float | 1.5 | Weight of the distribution focal loss. | Keep at default 1.5 for most cases; increase to 2.0-3.0 for highly overlapping objects requiring precise distribution. |
| `pose` | float | 12.0 | Weight of the pose loss in models trained for pose estimation. | Only relevant for pose models; increase to 15.0-20.0 for applications requiring very precise keypoint localization. |
| `kobj` | float | 2.0 | Weight of the keypoint objectness loss in pose estimation. | Only for pose models; balance with pose parameter (typically 1:5 to 1:8 ratio) depending on application needs. |
| `nbs` | int | 64 | Nominal batch size for normalization of loss. | Rarely needs changing; adjust only if experiencing unstable gradients with very large or small batch sizes. |
| `overlap_mask` | bool | True | Determines mask merging strategy for overlapping objects. | Keep True for most segmentation tasks; set False only for specialized cases with transparent overlapping objects. |
| `mask_ratio` | int | 4 | Downsample ratio for segmentation masks. | Use 4 for balance of speed and accuracy; lower (2) for fine detail preservation; higher (8) for speed on large masks. |
| `dropout` | float | 0.0 | Dropout rate for regularization in classification. | Set to 0.1-0.3 for classifiers prone to overfitting; keep at 0.0 for most detection tasks unless overfitting is observed. |
| `val` | bool | True | Enables validation during training. | Always keep True to monitor training progress; False only in rare cases where validation would be redundant. |
| `plots` | bool | False | Generates and saves plots of training metrics and examples. | Enable for visualizing training progress and model performance; especially useful for debugging or tuning hyperparameters. |

# YOLO Augmentation Parameters with Practical Guidelines

| Argument | Type | Default | Range | Description | Practical Guideline |
|----------|------|---------|-------|-------------|---------------------|
| `hsv_h` | float | 0.015 | 0.0 - 1.0 | Adjusts the hue of the image by a fraction of the color wheel, introducing color variability. Helps the model generalize across different lighting conditions. | Keep values small (0.015-0.05) for realistic colors; use higher values (0.1-0.2) only when color variation is critical for detection. |
| `hsv_s` | float | 0.7 | 0.0 - 1.0 | Alters the saturation of the image by a fraction, affecting the intensity of colors. Useful for simulating different environmental conditions. | Use higher values (0.5-0.7) for outdoor datasets with varying lighting conditions; lower (0.2-0.4) for controlled indoor environments. |
| `hsv_v` | float | 0.4 | 0.0 - 1.0 | Modifies the value (brightness) of the image by a fraction, helping the model to perform well under various lighting conditions. | Set to 0.3-0.5 for datasets with mixed lighting; increase to 0.6-0.8 for night/low-light detection scenarios. |
| `degrees` | float | 0.0 | 0.0 - 180 | Rotates the image randomly within the specified degree range, improving the model's ability to recognize objects at various orientations. | Use 0-10° for objects with directional properties (road markings, text); 10-30° for orientation-independent objects (people, animals); avoid for horizon-aligned scenes. |
| `translate` | float | 0.1 | 0.0 - 1.0 | Translates the image horizontally and vertically by a fraction of the image size, aiding in learning to detect partially visible objects. | Set to 0.1-0.2 for centered objects; increase to 0.3-0.5 for objects that appear at image edges or are partially visible. |
| `scale` | float | 0.5 | >=0.0 | Scales the image by a gain factor, simulating objects at different distances from the camera. | Use 0.4-0.6 for general detection tasks; higher (0.7-0.9) when object size varies significantly in your dataset. |
| `shear` | float | 0.0 | -180 - +180 | Shears the image by a specified degree, mimicking the effect of objects being viewed from different angles. | Keep low (1.0-3.0) for most tasks; increase to 5.0-10.0 for datasets with varied camera angles; use cautiously as it can distort object shapes. |
| `perspective` | float | 0.0 | 0.0 - 0.001 | Applies a random perspective transformation to the image, enhancing the model's ability to understand objects in 3D space. | Use very small values (0.0001-0.0005) to maintain realistic object appearances; higher values distort objects excessively. |
| `flipud` | float | 0.0 | 0.0 - 1.0 | Flips the image upside down with the specified probability, increasing the data variability without affecting the object's characteristics. | Generally keep at 0.0; only use (0.3-0.5) for orientation-invariant objects like aerial imagery or symmetric objects. |
| `fliplr` | float | 0.5 | 0.0 - 1.0 | Flips the image left to right with the specified probability, useful for learning symmetrical objects and increasing dataset diversity. | Set to 0.5 for most datasets; reduce to 0.0 for directional objects (text, arrows); essential for doubling effective dataset size. |
| `bgr` | float | 0.0 | 0.0 - 1.0 | Flips the image channels from RGB to BGR with the specified probability, useful for increasing robustness to incorrect channel ordering. | Usually keep at 0.0; set to 0.1-0.3 only when developing models that might be deployed across different image processing pipelines. |
| `mosaic` | float | 1.0 | 0.0 - 1.0 | Combines four training images into one, simulating different scene compositions and object interactions. | Set to 0.7-0.9 for small object detection tasks; reduce to 0.5-0.6 for datasets with thin or fine features (cracks, wires); disable (0.0) in the final few epochs. |
| `mixup` | float | 0.0 | 0.0 - 1.0 | Blends two images and their labels, creating a composite image. Enhances the model's ability to generalize by introducing label noise and visual variability. | Use 0.1-0.2 for small datasets; 0.3-0.4 for complex scenes; avoid or keep very low (0.05) for fine-detail detection tasks. |
| `cutmix` | float | 0.0 | 0.0 - 1.0 | Combines portions of two images, creating a partial blend while maintaining distinct regions. | Set to 0.2-0.3 for classification tasks; lower (0.1) for detection of small objects; higher (0.4-0.5) for robust multi-class detection. |
| `copy_paste` | float | 0.0 | 0.0 - 1.0 | *Segmentation only*. Copies and pastes objects across images to increase object instances. | Use 0.2-0.4 for rare classes with few samples; increase to 0.5-0.7 for extreme class imbalance; critical for improving recall of uncommon objects. |
| `copy_paste_mode` | str | flip | - | *Segmentation only*. Specifies the `copy-paste` strategy to use. Options include `'flip'` and `'mixup'`. | Use 'flip' for most segmentation tasks; try 'mixup' when diversity in object appearance is more important than position. |
| `auto_augment` | str | randaugment | - | *Classification only*. Applies a predefined augmentation policy to enhance model performance through visual diversity. | 'randaugment' works well for general tasks; try 'autoaugment' for known datasets (CIFAR, ImageNet); 'augmix' for highest robustness to distribution shifts. |
| `erasing` | float | 0.4 | 0.0 - 0.9 | *Classification only*. Randomly erases regions of the image during training to encourage the model to focus on less obvious features. | Set to 0.3-0.5 for general classification; increase to 0.6-0.8 for objects with distinctive parts that should not be relied on exclusively. |

## Recommended Parameters

### For General Object Detection
- `mosaic=0.8` - Strong but not overwhelming mosaic augmentation
- `mixup=0.1` - Light blending to maintain feature clarity
- `degrees=10.0` - Moderate rotation for orientation robustness
- `fliplr=0.5` - Standard horizontal flipping
- `translate=0.1` - Slight position variation
- `scale=0.5` - Balanced size variation
- `pretrained=True` - Use transfer learning from COCO dataset
- `batch=16` (or highest that fits on GPU)
- `imgsz=640`
- `epochs=100`
- `patience=50`
- `lr0=0.002`, `lrf=0.01`
- `cos_lr=True`
- `close_mosaic=10`

### For Small Object Detection (e.g., defects, cracks)
- `mosaic=0.6` - Lower mosaic to prevent tiny objects from disappearing
- `mixup=0.05` - Very light blending to preserve small features
- `degrees=5.0` - Minimal rotation
- `scale=0.6` - Emphasize varying sizes
- `hsv_v=0.5` - Enhanced brightness variation
- `copy_paste=0.3` - Increase instances of rare defects
- `imgsz=1280` (larger input size)
- `batch=8` (accommodate larger images)
- `box=10.0` (emphasize localization accuracy)
- `iou=0.35` (more lenient matching criteria)
- `epochs=150` (longer training)
- `close_mosaic=15` (more stabilization epochs)

### For Robust Real-World Deployment
- `hsv_h=0.02, hsv_s=0.7, hsv_v=0.4` - Comprehensive color variation
- `mosaic=0.7` - Balanced mosaic augmentation
- `mixup=0.2` - Moderate blending for robustness
- `translate=0.2` - Increased position variation
- `perspective=0.0003` - Slight perspective change
- `degrees=15.0` - Stronger rotation for viewpoint invariance
- Final training phase after main training:
  - `lr0=0.0001` (very low learning rate)
  - `epochs=20` (short fine-tuning)
  - `rect=True` (rectangular inference optimization)
  - `multi_scale=True` (robust to different image sizes)
  - `batch=4` (smallest stable batch size for best generalization)

### For Limited Dataset Training
- `mosaic=0.9` - Heavy mosaic to maximize data combination
- `mixup=0.2` - Moderate blending for artificial diversity
- `cutmix=0.2` - Additional variation technique
- `copy_paste=0.4` - Replicate scarce objects
- `fliplr=0.5, scale=0.7` - Maximize effective data variation